In [ ]:
## This file implements neural networks and logistic regression on p0006presabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 96.72% for combination data, and 88.46% for over-sampling data after improvememt.
## For logistic regression, the accuracy is 96.72% for combination data, and 75.96% for over-sampling data.
## We further construct random forest models, which are relatively less likely to bring overfitting 
## compared to decision tree.
## For random forest, the accuracy is 100% for combination data, and 87.5% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 97.86% for combination data, and 75.6% for over-sampling.

In [1]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0006presabs_quant.csv')
df.shape

(255, 86)

In [2]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [3]:
df['pheno']

0      0.537000
1      0.325833
2      0.630333
3      0.422500
4      0.433833
5      0.505833
6      0.431167
7      0.228333
8      0.547167
9      0.436667
10     0.366333
11     0.903500
12     0.431667
13     0.327500
14     0.339833
15     0.217000
16     0.441333
17     0.442667
18     0.794833
19     0.215000
20     0.426167
21     0.611667
22     0.322833
23     0.219667
24     1.004333
25     0.175000
26     0.837667
27     0.184667
28     0.195833
29     0.711667
         ...   
225    0.546400
226    0.545700
227    0.606100
228    0.964600
229    0.775600
230    0.441500
231    0.356167
232    0.437167
233    0.698667
234    0.455333
235    0.914500
236    0.231167
237    0.213667
238    0.341833
239    0.361667
240    0.342833
241    0.650667
242    0.287667
243    0.301167
244    0.514333
245    0.232500
246    0.429833
247    0.335667
248    0.191500
249    0.327667
250    0.449167
251    0.306167
252    0.388500
253    0.374833
254    0.330833
Name: pheno, Length: 255

In [4]:
df.head()

,id,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,...,group_7850,group_7871,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.537000
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.325833
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.630333
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.422500
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.433833


In [5]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [6]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [7]:
df['pheno']
df['pheno'].value_counts()

0    173
1     82
Name: pheno, dtype: int64

In [8]:
df.shape

(255, 86)

In [9]:
df_clean = df.drop(columns=['id'])

In [10]:
df_clean.shape

(255, 85)

In [11]:
df_clean.head()

,TTGTTATAGTC,TTAATTTAATAGA,TTAACATAATAAT,TGCAATCTCTTTAT,TATTATGTTAATG,TACATACCGAT,GTGTATCATAAT,GCTGTTGAAATGGC,GCAAACATGCG,GAGTCCTGTT,...,group_7850,group_7871,group_10187,group_10190,group_4016,group_6375,group_7207,group_8240,group_9605,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 84) (255,)


In [13]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 128), (1, 75)]


In [14]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 173), (1, 173)]


In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [17]:
############# Fully-Connected Neural Network ################

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [23]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [24]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 142 samples, validate on 61 samples
Epoch 1/100
142/142 [==============================] - 0s 762us/step - loss: 0.7570 - accuracy: 0.3310 - val_loss: 0.7500 - val_accuracy: 0.3279
Epoch 2/100
142/142 [==============================] - 0s 124us/step - loss: 0.7112 - accuracy: 0.4859 - val_loss: 0.7120 - val_accuracy: 0.4426
Epoch 3/100
142/142 [==============================] - 0s 179us/step - loss: 0.6756 - accuracy: 0.5352 - val_loss: 0.6838 - val_accuracy: 0.4426
Epoch 4/100
142/142 [==============================] - 0s 286us/step - loss: 0.6491 - accuracy: 0.6901 - val_loss: 0.6628 - val_accuracy: 0.6721
Epoch 5/100
142/142 [==============================] - 0s 122us/step - loss: 0.6289 - accuracy: 0.7676 - val_loss: 0.6463 - val_accuracy: 0.7049
Epoch 6/100
142/142 [==============================] - 0s 159us/step - loss: 0.6134 - accuracy: 0.7746 - val_loss: 0.6306 - val_accuracy: 0.7377
Epoch 7/100
142/142 [==============================] - 0s 160us/step - loss: 0.5971 -

142/142 [==============================] - 0s 177us/step - loss: 0.2918 - accuracy: 0.9577 - val_loss: 0.3202 - val_accuracy: 0.9508
Epoch 57/100
142/142 [==============================] - 0s 150us/step - loss: 0.2888 - accuracy: 0.9577 - val_loss: 0.3170 - val_accuracy: 0.9508
Epoch 58/100
142/142 [==============================] - 0s 177us/step - loss: 0.2862 - accuracy: 0.9577 - val_loss: 0.3140 - val_accuracy: 0.9508
Epoch 59/100
142/142 [==============================] - 0s 271us/step - loss: 0.2831 - accuracy: 0.9577 - val_loss: 0.3115 - val_accuracy: 0.9508
Epoch 60/100
142/142 [==============================] - 0s 150us/step - loss: 0.2804 - accuracy: 0.9577 - val_loss: 0.3089 - val_accuracy: 0.9508
Epoch 61/100
142/142 [==============================] - 0s 151us/step - loss: 0.2770 - accuracy: 0.9577 - val_loss: 0.3060 - val_accuracy: 0.9508
Epoch 62/100
142/142 [==============================] - 0s 198us/step - loss: 0.2746 - accuracy: 0.9648 - val_loss: 0.3034 - val_accuracy

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.107158). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


Epoch 73/100
142/142 [==============================] - 0s 280us/step - loss: 0.2487 - accuracy: 0.9718 - val_loss: 0.2784 - val_accuracy: 0.9508
Epoch 74/100
142/142 [==============================] - 0s 232us/step - loss: 0.2465 - accuracy: 0.9718 - val_loss: 0.2764 - val_accuracy: 0.9508
Epoch 75/100
142/142 [==============================] - 0s 203us/step - loss: 0.2444 - accuracy: 0.9718 - val_loss: 0.2745 - val_accuracy: 0.9508
Epoch 76/100
142/142 [==============================] - 0s 208us/step - loss: 0.2422 - accuracy: 0.9718 - val_loss: 0.2725 - val_accuracy: 0.9508
Epoch 77/100
142/142 [==============================] - 0s 223us/step - loss: 0.2404 - accuracy: 0.9718 - val_loss: 0.2706 - val_accuracy: 0.9508
Epoch 78/100
142/142 [==============================] - 0s 192us/step - loss: 0.2384 - accuracy: 0.9718 - val_loss: 0.2685 - val_accuracy: 0.9508
Epoch 79/100
142/142 [==============================] - 0s 209us/step - loss: 0.2365 - accuracy: 0.9718 - val_loss: 0.2664 -

In [26]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

61/61 [==============================] - 0s 138us/step
combination test accuracy: 96.72%


In [93]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [94]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [95]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 242 samples, validate on 104 samples
Epoch 1/100
242/242 [==============================] - 0s 1ms/step - loss: 0.7374 - accuracy: 0.4917 - val_loss: 0.7121 - val_accuracy: 0.5192
Epoch 2/100
242/242 [==============================] - 0s 137us/step - loss: 0.7233 - accuracy: 0.4876 - val_loss: 0.7009 - val_accuracy: 0.5192
Epoch 3/100
242/242 [==============================] - 0s 145us/step - loss: 0.7117 - accuracy: 0.5083 - val_loss: 0.6921 - val_accuracy: 0.5577
Epoch 4/100
242/242 [==============================] - 0s 561us/step - loss: 0.7007 - accuracy: 0.5537 - val_loss: 0.6839 - val_accuracy: 0.5769
Epoch 5/100
242/242 [==============================] - 0s 126us/step - loss: 0.6920 - accuracy: 0.5372 - val_loss: 0.6764 - val_accuracy: 0.5673
Epoch 6/100
242/242 [==============================] - 0s 170us/step - loss: 0.6833 - accuracy: 0.5413 - val_loss: 0.6698 - val_accuracy: 0.6058
Epoch 7/100
242/242 [==============================] - 0s 190us/step - loss: 0.6751 - 

Epoch 57/100
242/242 [==============================] - 0s 96us/step - loss: 0.5081 - accuracy: 0.7686 - val_loss: 0.5201 - val_accuracy: 0.7788
Epoch 58/100
242/242 [==============================] - 0s 832us/step - loss: 0.5075 - accuracy: 0.7810 - val_loss: 0.5193 - val_accuracy: 0.7692
Epoch 59/100
242/242 [==============================] - 0s 177us/step - loss: 0.5057 - accuracy: 0.7727 - val_loss: 0.5190 - val_accuracy: 0.7500
Epoch 60/100
242/242 [==============================] - 0s 139us/step - loss: 0.5051 - accuracy: 0.7851 - val_loss: 0.5174 - val_accuracy: 0.7500
Epoch 61/100
242/242 [==============================] - 0s 149us/step - loss: 0.5034 - accuracy: 0.7810 - val_loss: 0.5164 - val_accuracy: 0.7500
Epoch 62/100
242/242 [==============================] - 0s 189us/step - loss: 0.5027 - accuracy: 0.7851 - val_loss: 0.5155 - val_accuracy: 0.7788
Epoch 63/100
242/242 [==============================] - 0s 158us/step - loss: 0.5016 - accuracy: 0.7810 - val_loss: 0.5148 - 

In [96]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

104/104 [==============================] - 0s 86us/step
over-sampling test accuracy: 76.92%


In [97]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [98]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [99]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 242 samples, validate on 104 samples
Epoch 1/1000
242/242 [==============================] - 0s 595us/step - loss: 0.6984 - accuracy: 0.5083 - val_loss: 0.7006 - val_accuracy: 0.4519
Epoch 2/1000
242/242 [==============================] - 0s 120us/step - loss: 0.6941 - accuracy: 0.5124 - val_loss: 0.6973 - val_accuracy: 0.5000
Epoch 3/1000
242/242 [==============================] - 0s 126us/step - loss: 0.6902 - accuracy: 0.5413 - val_loss: 0.6942 - val_accuracy: 0.5288
Epoch 4/1000
242/242 [==============================] - 0s 189us/step - loss: 0.6857 - accuracy: 0.5702 - val_loss: 0.6913 - val_accuracy: 0.5385
Epoch 5/1000
242/242 [==============================] - 0s 135us/step - loss: 0.6819 - accuracy: 0.5744 - val_loss: 0.6886 - val_accuracy: 0.5673
Epoch 6/1000
242/242 [==============================] - 0s 117us/step - loss: 0.6788 - accuracy: 0.5950 - val_loss: 0.6858 - val_accuracy: 0.5962
Epoch 7/1000
242/242 [==============================] - 0s 592us/step - loss: 

Epoch 56/1000
242/242 [==============================] - 0s 117us/step - loss: 0.5506 - accuracy: 0.6983 - val_loss: 0.5753 - val_accuracy: 0.6827
Epoch 57/1000
242/242 [==============================] - 0s 117us/step - loss: 0.5488 - accuracy: 0.6942 - val_loss: 0.5733 - val_accuracy: 0.6731
Epoch 58/1000
242/242 [==============================] - 0s 109us/step - loss: 0.5461 - accuracy: 0.6983 - val_loss: 0.5712 - val_accuracy: 0.6635
Epoch 59/1000
242/242 [==============================] - 0s 103us/step - loss: 0.5435 - accuracy: 0.6942 - val_loss: 0.5692 - val_accuracy: 0.6731
Epoch 60/1000
242/242 [==============================] - 0s 117us/step - loss: 0.5417 - accuracy: 0.6983 - val_loss: 0.5672 - val_accuracy: 0.6827
Epoch 61/1000
242/242 [==============================] - 0s 118us/step - loss: 0.5392 - accuracy: 0.6983 - val_loss: 0.5653 - val_accuracy: 0.6923
Epoch 62/1000
242/242 [==============================] - 0s 91us/step - loss: 0.5371 - accuracy: 0.6983 - val_loss: 0.

242/242 [==============================] - 0s 95us/step - loss: 0.4659 - accuracy: 0.7727 - val_loss: 0.5035 - val_accuracy: 0.7500
Epoch 112/1000
242/242 [==============================] - 0s 116us/step - loss: 0.4652 - accuracy: 0.7686 - val_loss: 0.4995 - val_accuracy: 0.7404
Epoch 113/1000
242/242 [==============================] - 0s 105us/step - loss: 0.4644 - accuracy: 0.7727 - val_loss: 0.4999 - val_accuracy: 0.7404
Epoch 114/1000
242/242 [==============================] - 0s 515us/step - loss: 0.4645 - accuracy: 0.7645 - val_loss: 0.5013 - val_accuracy: 0.7500
Epoch 115/1000
242/242 [==============================] - 0s 141us/step - loss: 0.4631 - accuracy: 0.7686 - val_loss: 0.5016 - val_accuracy: 0.7596
Epoch 116/1000
242/242 [==============================] - 0s 128us/step - loss: 0.4626 - accuracy: 0.7686 - val_loss: 0.4935 - val_accuracy: 0.7404
Epoch 117/1000
242/242 [==============================] - 0s 180us/step - loss: 0.4619 - accuracy: 0.7893 - val_loss: 0.4951 - v

242/242 [==============================] - 0s 109us/step - loss: 0.4338 - accuracy: 0.8017 - val_loss: 0.4753 - val_accuracy: 0.7500
Epoch 167/1000
242/242 [==============================] - 0s 108us/step - loss: 0.4320 - accuracy: 0.8058 - val_loss: 0.4775 - val_accuracy: 0.7500
Epoch 168/1000
242/242 [==============================] - 0s 127us/step - loss: 0.4322 - accuracy: 0.8017 - val_loss: 0.4790 - val_accuracy: 0.7500
Epoch 169/1000
242/242 [==============================] - 0s 150us/step - loss: 0.4315 - accuracy: 0.7975 - val_loss: 0.4724 - val_accuracy: 0.7500
Epoch 170/1000
242/242 [==============================] - 0s 117us/step - loss: 0.4314 - accuracy: 0.7975 - val_loss: 0.4718 - val_accuracy: 0.7500
Epoch 171/1000
242/242 [==============================] - 0s 114us/step - loss: 0.4300 - accuracy: 0.7975 - val_loss: 0.4756 - val_accuracy: 0.7500
Epoch 172/1000
242/242 [==============================] - 0s 168us/step - loss: 0.4293 - accuracy: 0.7975 - val_loss: 0.4711 - 

Epoch 221/1000
242/242 [==============================] - 0s 114us/step - loss: 0.4114 - accuracy: 0.8140 - val_loss: 0.4586 - val_accuracy: 0.7500
Epoch 222/1000
242/242 [==============================] - 0s 99us/step - loss: 0.4114 - accuracy: 0.8058 - val_loss: 0.4609 - val_accuracy: 0.7500
Epoch 223/1000
242/242 [==============================] - 0s 106us/step - loss: 0.4126 - accuracy: 0.8058 - val_loss: 0.4602 - val_accuracy: 0.7500
Epoch 224/1000
242/242 [==============================] - 0s 109us/step - loss: 0.4106 - accuracy: 0.8140 - val_loss: 0.4615 - val_accuracy: 0.7500
Epoch 225/1000
242/242 [==============================] - 0s 121us/step - loss: 0.4103 - accuracy: 0.8140 - val_loss: 0.4632 - val_accuracy: 0.7500
Epoch 226/1000
242/242 [==============================] - 0s 190us/step - loss: 0.4098 - accuracy: 0.8223 - val_loss: 0.4595 - val_accuracy: 0.7500
Epoch 227/1000
242/242 [==============================] - 0s 163us/step - loss: 0.4096 - accuracy: 0.8099 - val_l

...

242/242 [==============================] - 0s 883us/step - loss: 0.2877 - accuracy: 0.8760 - val_loss: 0.3892 - val_accuracy: 0.8077
Epoch 761/1000
242/242 [==============================] - 0s 171us/step - loss: 0.2828 - accuracy: 0.8678 - val_loss: 0.3915 - val_accuracy: 0.8077
Epoch 762/1000
242/242 [==============================] - 0s 173us/step - loss: 0.2850 - accuracy: 0.8471 - val_loss: 0.3896 - val_accuracy: 0.8077
Epoch 763/1000
242/242 [==============================] - 0s 234us/step - loss: 0.2872 - accuracy: 0.8512 - val_loss: 0.4114 - val_accuracy: 0.8269
Epoch 764/1000
242/242 [==============================] - 0s 327us/step - loss: 0.2863 - accuracy: 0.8760 - val_loss: 0.3896 - val_accuracy: 0.8365
Epoch 765/1000
242/242 [==============================] - 0s 166us/step - loss: 0.2791 - accuracy: 0.8760 - val_loss: 0.3899 - val_accuracy: 0.8077
Epoch 766/1000
242/242 [==============================] - 0s 135us/step - loss: 0.2833 - accuracy: 0.8678 - val_loss: 0.4016 - 

242/242 [==============================] - 0s 152us/step - loss: 0.2773 - accuracy: 0.8760 - val_loss: 0.3898 - val_accuracy: 0.8365
Epoch 816/1000
242/242 [==============================] - 0s 123us/step - loss: 0.2828 - accuracy: 0.8595 - val_loss: 0.3970 - val_accuracy: 0.8173
Epoch 817/1000
242/242 [==============================] - 0s 143us/step - loss: 0.2760 - accuracy: 0.8760 - val_loss: 0.3930 - val_accuracy: 0.8077
Epoch 818/1000
242/242 [==============================] - 0s 98us/step - loss: 0.2744 - accuracy: 0.8719 - val_loss: 0.3921 - val_accuracy: 0.8269
Epoch 819/1000
242/242 [==============================] - ETA: 0s - loss: 0.2773 - accuracy: 0.87 - 0s 193us/step - loss: 0.2745 - accuracy: 0.8760 - val_loss: 0.3934 - val_accuracy: 0.8077
Epoch 820/1000
242/242 [==============================] - 0s 134us/step - loss: 0.2786 - accuracy: 0.8595 - val_loss: 0.4167 - val_accuracy: 0.8462
Epoch 821/1000
242/242 [==============================] - 0s 292us/step - loss: 0.2732

242/242 [==============================] - 0s 116us/step - loss: 0.2723 - accuracy: 0.8678 - val_loss: 0.4051 - val_accuracy: 0.8365
Epoch 871/1000
242/242 [==============================] - 0s 122us/step - loss: 0.2872 - accuracy: 0.8760 - val_loss: 0.3922 - val_accuracy: 0.8077
Epoch 872/1000
242/242 [==============================] - 0s 100us/step - loss: 0.2704 - accuracy: 0.8636 - val_loss: 0.3911 - val_accuracy: 0.8365
Epoch 873/1000
242/242 [==============================] - 0s 118us/step - loss: 0.2672 - accuracy: 0.8719 - val_loss: 0.3934 - val_accuracy: 0.8173
Epoch 874/1000
242/242 [==============================] - 0s 164us/step - loss: 0.2789 - accuracy: 0.8636 - val_loss: 0.3987 - val_accuracy: 0.8750
Epoch 875/1000
242/242 [==============================] - 0s 122us/step - loss: 0.2739 - accuracy: 0.8760 - val_loss: 0.3919 - val_accuracy: 0.8462
Epoch 876/1000
242/242 [==============================] - 0s 167us/step - loss: 0.2716 - accuracy: 0.8802 - val_loss: 0.3949 - 

242/242 [==============================] - 0s 122us/step - loss: 0.2796 - accuracy: 0.8554 - val_loss: 0.3954 - val_accuracy: 0.8365
Epoch 926/1000
242/242 [==============================] - 0s 151us/step - loss: 0.2722 - accuracy: 0.8636 - val_loss: 0.4139 - val_accuracy: 0.8558
Epoch 927/1000
242/242 [==============================] - 0s 101us/step - loss: 0.2587 - accuracy: 0.8843 - val_loss: 0.4509 - val_accuracy: 0.7692
Epoch 928/1000
242/242 [==============================] - 0s 102us/step - loss: 0.2733 - accuracy: 0.8636 - val_loss: 0.4019 - val_accuracy: 0.8269
Epoch 929/1000
242/242 [==============================] - 0s 132us/step - loss: 0.2675 - accuracy: 0.8760 - val_loss: 0.4050 - val_accuracy: 0.8750
Epoch 930/1000
242/242 [==============================] - 0s 119us/step - loss: 0.2706 - accuracy: 0.8636 - val_loss: 0.3971 - val_accuracy: 0.8365
Epoch 931/1000
242/242 [==============================] - 0s 191us/step - loss: 0.2726 - accuracy: 0.8678 - val_loss: 0.3977 - 

242/242 [==============================] - 0s 115us/step - loss: 0.2625 - accuracy: 0.8636 - val_loss: 0.4163 - val_accuracy: 0.8750
Epoch 981/1000
242/242 [==============================] - 0s 103us/step - loss: 0.2649 - accuracy: 0.8719 - val_loss: 0.3984 - val_accuracy: 0.8846
Epoch 982/1000
242/242 [==============================] - 0s 109us/step - loss: 0.2607 - accuracy: 0.8719 - val_loss: 0.4016 - val_accuracy: 0.8269
Epoch 983/1000
242/242 [==============================] - 0s 109us/step - loss: 0.2633 - accuracy: 0.8760 - val_loss: 0.4123 - val_accuracy: 0.8750
Epoch 984/1000
242/242 [==============================] - 0s 101us/step - loss: 0.2680 - accuracy: 0.8802 - val_loss: 0.3992 - val_accuracy: 0.8462
Epoch 985/1000
242/242 [==============================] - 0s 117us/step - loss: 0.2635 - accuracy: 0.8760 - val_loss: 0.3995 - val_accuracy: 0.8173
Epoch 986/1000
242/242 [==============================] - 0s 97us/step - loss: 0.2753 - accuracy: 0.8719 - val_loss: 0.3991 - v

In [100]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

104/104 [==============================] - 0s 73us/step
over-sampling test accuracy: 88.46%


In [ ]:
############ Logistic Regression ############

In [101]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [102]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [103]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 96.72%


In [104]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [105]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 75.96%


In [106]:
############## Random Forest ##############

In [107]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [108]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [109]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [110]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 87.50%


In [111]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [112]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.92857143 1.         0.96428571]
0.9785714285714286


In [113]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.78       0.83333333 0.70833333 0.6875     0.77083333]
0.756
